Before this, you need to install PynPoint version 0.8.1 by way of : pip install install pynpoint==0.8.1 and trackpy by: 
pip install trackpy in your anaconda terminal.

$\textbf{Importing the necessary modules from PynPoint, together with necessary python modules for plotting.}$

In [5]:
import matplotlib.pyplot as plt
import os
import numpy as np
from astropy.visualization import astropy_mpl_style
from matplotlib.patches import Circle
from astropy import table
from astropy.utils.data import get_pkg_data_filename
from matplotlib.cm import ScalarMappable
from Sep_PA import calc_sep_pa3
from mpl_toolkits.axes_grid1 import make_axes_locatable

from ImageFunctions import UnsharpMaskModule, CorrectDistortionModule_SPHERE
from BadPixelCleaning import BadPixelMapModule_IRDIS
from pynpoint import Pypeline, FitsReadingModule, FitsWritingModule, StackCubesModule, FlatCalibrationModule, \
    BadPixelSigmaFilterModule, NoddingBackgroundModule, AngleCalculationModule, CombineTagsModule, \
    SortParangModule, StackAndSubsetModule, CropImagesModule, DerotateAndStackModule, ReplaceBadPixelsModule, \
    WaffleCenteringModule, PSFpreparationModule, PcaPsfSubtractionModule, ClassicalADIModule, AttributeWritingModule, \
    DarkCalibrationModule, BadPixelMapModule, AddLinesModule, Hdf5WritingModule, Hdf5ReadingModule, FitCenterModule

#if you get a warning from jupyter here, you might need to downgrade to an older version op ipython by starting up an
#anaconda terminal and running : pip install install ipython==7.10.0

$\textbf{Parameters}$

In [4]:
target = "" #here just put the name of the observed target

q

filt = "B_H"
instrument = "SPHERE"
instrument_det = "IRDIS"

path = ""#this should be the path of the target, in this target folder should be an output folder, an input folder and the
#four other python files containing the necessary functions, same path as for the master flat file.


pixscale = 0.01227 #there three sizes are the same for all SPHERE images.

im_size_1 = 1031
im_size_2 = 1023
im_size_3 = 201


star_1_pos = (,) #these two parameters are the x and y positions (centres) of the stars in the left and right frame of a sphere image.
#one of the functions (WaffleCenteringModule()) needs these to make sure both sides are added together on the correct positions.
#to get these you need to look at a raw image in SAOImageDS9 and take the positions from there (open the fits file). If that 
#does not result in correct combination of both sides, add the write_addlines_arr() module and look at its output .fits file
#and take the centres from there.
star_2_pos = (,)
radius = #for this you need to look at the centre frames from SPHERE and apply a circular region over the star in the frames
#in DS9, this region needs to just barely contain the 'Waffles' around the star, the radius of this region is what 
#you should put here.
kernel_size = 5 #stays the same for all SPHERE images.

SyntaxError: invalid syntax (<ipython-input-4-feb4c96b2ea8>, line 22)

$\textbf{Define directories}$

In [ ]:
working_place_in = os.path.join(path)
input_place_in = os.path.join(path,'input\\') #the directory in which all your files are. Within this directory
#the files need to be sorted based on nature: dark, sky, science, center, flat etc.
output_place_in = os.path.join(path,'output\\')#here the master flat
#will be put after its creation, which you can then use again for the data reduction

$\textbf{Configuring directories}$

In [ ]:
# Create Working directory for this target
if not os.path.isdir(working_place_in):
    print("Creating directory: %s" % (working_place_in))
    os.makedirs(working_place_in)

# Create directory for results
if not os.path.isdir(output_place_in):
    print("Creating directory: %s" % (output_place_in))
    os.makedirs(output_place_in)

pipeline = Pypeline(working_place_in=path,
                    input_place_in,
                    output_place_in)

$\textbf{Reading the science data}$

In [ ]:
science = FitsReadingModule(name_in='read_science',
                           input_dir=input_place_in + 'SCIENCE',
                           image_tag='im_arr',
                           overwrite=True,
                           check=False,)

$\textbf{Reading the dark data}$

In [ ]:
dark = FitsReadingModule(name_in='read_dark',
                           input_dir=input_place_in + 'DARK',
                           image_tag='dark_arr',
                           overwrite=True,
                           check=False,)

$\textbf{Reading the master flat data}$

In [ ]:
flat = FitsReadingModule(name_in='flat',
                           input_dir=output_place_in + 'master_Flat',
                           image_tag='master_flat_arr',
                           overwrite=True,
                           check=False,)

$\textbf{Reading the sky data}$

In [ ]:
sky = FitsReadingModule(name_in="sky_reading",
                                input_dir=input_place_in + 'SKY',
                                image_tag="sky_arr")

$\textbf{Reading the center data}$

In [ ]:
center_reading = FitsReadingModule(name_in="center_reading",
                                   input_dir=input_place_in + 'CENTER',
                                   image_tag="center_arr")

$\textbf{Calculate parallactic angle}$

In [ ]:
angle_calculation = AngleCalculationModule(instrument="SPHERE/IRDIS",
                                           name_in="angle_calculation",
                                           data_tag="im_arr")

write_parang = AttributeWritingModule(name_in="write_parang",
                                      data_tag='im_arr',
                                      attribute="PARANG",
                                      file_name='parang.dat')

$\textbf{Median combine all the sky images}$

In [ ]:
sky_median = DerotateAndStackModule(name_in="median_sky_arr",
                                    image_in_tag="sky_arr",
                                    image_out_tag="sky_arr_median",
                                    derotate=False,
                                    stack='median',
                                    extra_rot=0.)

$\textbf{Substract sky from the science data}$

In [ ]:
sky_subtraction_science = DarkCalibrationModule(name_in="sky_subtraction_science",
                                                dark_in_tag="sky_arr_median",
                                                image_in_tag="im_arr",
                                                image_out_tag="im_arr_sub")

write_im_arr_sub = FitsWritingModule(file_name="im_arr_sub.fits",
                                     name_in="write_im_arr_sub",
                                     data_tag="im_arr_sub")

$\textbf{Substract sky from the center frame}$

In [ ]:
sky_subtraction_center = DarkCalibrationModule(name_in="sky_subtraction_center",
                                               dark_in_tag="sky_arr_median",
                                               image_in_tag="center_arr",
                                               image_out_tag="center_arr_sub")

write_center_arr_sub = FitsWritingModule(file_name="center_arr_sub.fits",
                                         name_in="write_center_arr_sub",
                                         data_tag="center_arr_sub")

$\textbf{Flat calibration science images}$

In [ ]:
flat_calibration_science = FlatCalibrationModule(name_in="flat_calibration_science",
                                                 image_in_tag="im_arr_sub",
                                                 flat_in_tag="master_flat_arr",
                                                 image_out_tag="im_arr_clean")

write_im_arr_clean = FitsWritingModule(file_name="im_arr_clean.fits",
                                       name_in="write_im_arr_clean",
                                       data_tag="im_arr_clean")

$\textbf{Flat calibration center frames}$

In [ ]:
flat_calibration_center = FlatCalibrationModule(name_in="flat_calibration_center",
                                                image_in_tag="center_arr_sub",
                                                flat_in_tag="master_flat_arr",
                                                image_out_tag="center_arr_clean")

write_center_arr_clean = FitsWritingModule(file_name="center_arr_clean.fits",
                                           name_in="write_center_arr_clean",
                                           data_tag="center_arr_clean")

$\textbf{Create a bad pixel map}$

In [ ]:
create_bp_map = BadPixelMapModule_IRDIS(name_in="create_bp_map",
                                        dark_in_tag="dark_arr",
                                        flat_in_tag="master_flat_arr",
                                        bp_map_out_tag="bp_map",
                                        dark_threshold=0.2,
                                        flat_threshold=0.2)

write_bp_map = FitsWritingModule(file_name="bp_map.fits",
                                 name_in="write_bp_map",
                                 data_tag="bp_map")

$\textbf{Run the bad pixel cleaning for the science data}$

In [ ]:
bp_cleaning_science_1 = ReplaceBadPixelsModule(name_in="bad_pixel_cleaning_science_1",
                                               image_in_tag="im_arr_clean",
                                               map_in_tag="bp_map",
                                               image_out_tag="im_arr_bp_clean_1",
                                               size=2,
                                               replace="mean")

write_im_arr_bp_clean_1 = FitsWritingModule(file_name="im_arr_bp_clean_1.fits",
                                            name_in="write_im_arr_bp_clean_1",
                                            data_tag="im_arr_bp_clean_1")

bp_cleaning_science_2 = BadPixelSigmaFilterModule(name_in="bad_pixel_cleaning_science_2",
                                                  image_in_tag="im_arr_bp_clean_1",
                                                  image_out_tag="im_arr_bp_clean_2",
                                                  box=9,
                                                  sigma=5.)

write_im_arr_bp_clean_2 = FitsWritingModule(file_name="im_arr_bp_clean_2.fits",
                                          name_in="write_im_arr_bp_clean_2",
                                          data_tag="im_arr_bp_clean_2")


$\textbf{Run the bad pixel cleaning on the reduced calibration data}$

In [ ]:
bp_cleaning_center = ReplaceBadPixelsModule(name_in="bad_pixel_cleaning_center",
                                            image_in_tag="center_arr_clean",
                                            map_in_tag="bp_map",
                                            image_out_tag="center_arr_bp_clean",
                                            size=2,
                                            replace="mean")

write_center_arr_bp_clean = FitsWritingModule(file_name="center_arr_bp_clean.fits",
                                              name_in="write_center_arr_bp_clean",
                                              data_tag="center_arr_bp_clean")

$\textbf{Correct for the distortion in the y direction}$

In [ ]:
correct_distortion_y_science = CorrectDistortionModule_SPHERE(name_in="correct_distortion_y_science",
                                                      image_in_tag="im_arr_bp_clean_2",
                                                      image_out_tag="im_arr_corrected")

correct_distortion_y_center = CorrectDistortionModule_SPHERE(name_in="correct_distortion_y_center",
                                                      image_in_tag="center_arr_bp_clean",
                                                      image_out_tag="center_arr_corrected")


$\textbf{Add lines so large image cutout is possible}$

In [ ]:
add_lines_science = AddLinesModule(name_in='add_lines_science',
                                   image_in_tag='im_arr_corrected',
                                   image_out_tag='im_arr_add',
                                   lines=(40,0,0,20))

add_lines_center = AddLinesModule(name_in='add_lines_center',
                                   image_in_tag='center_arr_corrected',
                                   image_out_tag='center_arr_add',
                                   lines=(40,0,0,20))

write_addlines_arr = FitsWritingModule(file_name="im_arr_add.fits",
                                           name_in="write_addlines_arr",
                                           data_tag="im_arr_add")

$\textbf{Center the images on the left side}$

In [ ]:
center_science_1 = WaffleCenteringModule(name_in="centering_images_1",
                                         image_in_tag="im_arr_add",
                                         center_in_tag="center_arr_add",
                                         image_out_tag="im_arr_centered_cut_1",
                                         size=im_size_1*pixscale,
                                         center=star_1_pos,
                                         radius=radius,
                                         pattern="x",
                                         sigma=5.*pixscale,
                                         dither=True) #here you need the x,y positions and radius of the star (centre) to add to the 
                                                        #module. Get this in the way described in the parameter cell.

write_im_arr_centered_cut_1 = FitsWritingModule(file_name="im_arr_centered_cut_1.fits",
                                                name_in="write_im_arr_centered_cut_1",
                                                data_tag="im_arr_centered_cut_1")

$\textbf{Center the images on the right side}$

In [ ]:
center_science_2 = WaffleCenteringModule(name_in="centering_images_2",
                                         image_in_tag="im_arr_add",
                                         center_in_tag="center_arr_add",
                                         image_out_tag="im_arr_centered_cut_2",
                                         size=im_size_1*pixscale,
                                         center=star_2_pos,
                                         radius=radius,
                                         pattern="x",
                                         sigma=5.*pixscale,
                                         dither=True)#same as in above cell, look at centre frame and raw science frame in DS9

write_im_arr_centered_cut_2 = FitsWritingModule(file_name="im_arr_centered_cut_2.fits",
                                                name_in="write_im_arr_centered_cut_2",
                                                data_tag="im_arr_centered_cut_2")

$\textbf{Merge both sides}$

In [ ]:
merge_left_and_right = CombineTagsModule(image_in_tags=["im_arr_centered_cut_1","im_arr_centered_cut_2"],
                                         check_attr=True,
                                         name_in="combine_left_and_right",
                                         image_out_tag="im_arr_centered_cut")

write_im_arr_centered_cut = FitsWritingModule(file_name="im_arr_centered_cut.fits",
                                              name_in="write_im_arr_centered_cut",
                                              data_tag="im_arr_centered_cut")

$\textbf{Sort the images for parallactic angle}$

In [ ]:
sort_parang = SortParangModule(name_in="sort_parang",
                               image_in_tag="im_arr_centered_cut",
                               image_out_tag="im_arr_sorted")

write_im_arr_sorted = FitsWritingModule(file_name="im_arr_sorted.fits",
                                        name_in="write_im_arr_sorted",
                                        data_tag="im_arr_sorted")

$\textbf{Mean combine left and right frames}$

In [ ]:
mean_left_right = StackAndSubsetModule(name_in="mean_left_right",
                                       image_in_tag="im_arr_sorted",
                                       image_out_tag="im_arr_final",
                                       stacking=2)

write_im_arr_final = FitsWritingModule(file_name="im_arr_final.fits",
                                       name_in="write_im_arr_final",
                                       data_tag="im_arr_final")

$\textbf{Cut images to final size}$

In [ ]:
cut_im_arr_final = CropImagesModule(size=im_size_2*pixscale,
                                    center=None,
                                    name_in="cut_im_arr_final",
                                    image_in_tag="im_arr_final",
                                    image_out_tag="im_arr_final_cut")

write_im_arr_final_cut = FitsWritingModule(file_name="%s_%s_arr_final_cut.fits"%(target,filt),
                                           name_in="write_im_arr_final_cut",
                                           data_tag="im_arr_final_cut", overwrite = True)

$\textbf{Create residuals}$

In [ ]:
create_residuals_0 = DerotateAndStackModule(name_in="create_residuals_0",
                                            image_in_tag="im_arr_final_cut",
                                            image_out_tag="res_rot",
                                            derotate=True,
                                            stack=None,
                                            extra_rot=0.)

write_res_rot = FitsWritingModule(file_name="%s_%s_res_rot.fits"%(target,filt),
                                   name_in="write_res_rot",
                                   data_tag="res_rot", overwrite = True)


$\textbf{Calculate the median of the derotated stack}$

In [ ]:
create_median = DerotateAndStackModule(name_in="create_median",
                                       image_in_tag="im_arr_final_cut",
                                       image_out_tag="res_median",
                                       derotate=True,
                                       stack="median",
                                       extra_rot=0.)

write_res_median = FitsWritingModule(file_name="%s_%s_res_median.fits"%(target,filt),
                                     name_in="write_res_median",
                                     data_tag="res_median", overwrite = True)

unsharp_mask_res_median = UnsharpMaskModule(name_in="unsharp_mask_res_median",
                                            image_in_tag="res_median",
                                            image_out_tag="res_median_unsharp",
                                            kernel_size=kernel_size)

write_res_median_unsharp = FitsWritingModule(file_name="%s_%s_res_median_unsharp.fits"%(target,filt),
                                             name_in="write_res_median_unsharp",
                                             data_tag="res_median_unsharp", overwrite = True)

$\textbf{Derotate and average}$

In [ ]:
create_residuals_1 = DerotateAndStackModule(name_in="create_residuals_1",
                                            image_in_tag="im_arr_final_cut",
                                            image_out_tag="res_mean",
                                            derotate=True,
                                            stack="mean")

write_res_mean = FitsWritingModule(file_name="%s_%s_res_mean.fits"%(target,filt),
                                   name_in="write_res_mean",
                                   data_tag="res_mean", overwrite = True)

unsharp_mask_res_mean = UnsharpMaskModule(name_in="apply_unsharp_mask_res_mean",
                                          image_in_tag="res_mean",
                                          image_out_tag="res_mean_unsharp",
                                          kernel_size=kernel_size)

write_res_mean_unsharp = FitsWritingModule(file_name="%s_%s_res_mean_unsharp.fits"%(target,filt),
                                           name_in="write_res_mean_unsharp",
                                           data_tag="res_mean_unsharp", overwrite = True)

$\textbf{cADI}$

In [ ]:
cADI_psf_sub_1 = ClassicalADIModule(threshold=None,
                                  nreference=None,
                                  residuals="median",
                                  extra_rot=0.,
                                  name_in="cADI_psf_sub_1",
                                  image_in_tag="im_arr_final_cut",
                                  res_out_tag="cADI_res_rot",
                                  stack_out_tag="cADI_res_median")

write_cADI_res_median = FitsWritingModule(file_name="%s_%s_cADI_res_median.fits" % (target, filt),
                                          name_in="write_cADI_res_median",
                                          data_tag="cADI_res_median", overwrite = True)

cADI_psf_sub_2 = ClassicalADIModule(threshold=None,
                                  nreference=None,
                                  residuals="mean",
                                  extra_rot=0.,
                                  name_in="cADI_psf_sub_2",
                                  image_in_tag="im_arr_final_cut",
                                  res_out_tag="cADI_res_rot",
                                  stack_out_tag="cADI_res_mean")

write_cADI_res_mean = FitsWritingModule(file_name="%s_%s_cADI_res_mean.fits" % (target, filt),
                                          name_in="write_cADI_res_mean",
                                          data_tag="cADI_res_mean", overwrite = True)

$\textbf{Adding all the modules to the pipeline}$

In [ ]:
pipeline.add_module(science)
pipeline.add_module(sky)
pipeline.add_module(dark)
pipeline.add_module(flat)
pipeline.add_module(center_reading)
pipeline.add_module(angle_calculation)
pipeline.add_module(write_parang)
pipeline.add_module(sky_median)
pipeline.add_module(sky_subtraction_science)
# pipeline.add_module(write_im_arr_sub)
pipeline.add_module(sky_subtraction_center)
# pipeline.add_module(write_center_arr_sub)
pipeline.add_module(flat_calibration_science)
# pipeline.add_module(write_im_arr_clean)
pipeline.add_module(flat_calibration_center)
# pipeline.add_module(write_center_arr_clean)
pipeline.add_module(create_bp_map)
# pipeline.add_module(write_bp_map)
pipeline.add_module(bp_cleaning_science_1)
# pipeline.add_module(write_im_arr_bp_clean_1)
pipeline.add_module(bp_cleaning_science_2)
# pipeline.add_module(write_im_arr_bp_clean_2)
pipeline.add_module(bp_cleaning_center)
pipeline.add_module(write_center_arr_bp_clean)
pipeline.add_module(correct_distortion_y_science)
pipeline.add_module(correct_distortion_y_center)
pipeline.add_module(add_lines_science)
pipeline.add_module(add_lines_center)
# pipeline.add_module(write_addlines_arr)
pipeline.add_module(center_science_1)
pipeline.add_module(write_im_arr_centered_cut_1)
pipeline.add_module(center_science_2)
pipeline.add_module(write_im_arr_centered_cut_2)
pipeline.add_module(merge_left_and_right)
pipeline.add_module(write_im_arr_centered_cut)
pipeline.add_module(sort_parang)
pipeline.add_module(write_im_arr_sorted)
pipeline.add_module(mean_left_right)
pipeline.add_module(write_im_arr_final)
pipeline.add_module(cut_im_arr_final)
pipeline.add_module(write_im_arr_final_cut)
pipeline.add_module(create_residuals_0)
pipeline.add_module(write_res_rot)
pipeline.add_module(create_median)
pipeline.add_module(write_res_median)
pipeline.add_module(unsharp_mask_res_median)
pipeline.add_module(write_res_median_unsharp)
pipeline.add_module(create_residuals_1)
pipeline.add_module(write_res_mean)
pipeline.add_module(unsharp_mask_res_mean)
pipeline.add_module(write_res_mean_unsharp)
pipeline.add_module(cADI_psf_sub_1)
pipeline.add_module(write_cADI_res_median)
pipeline.add_module(cADI_psf_sub_2)
pipeline.add_module(write_cADI_res_mean)

In [ ]:
pipeline.run()

$\textbf{Making the plots}$

In [ ]:
dat = get_pkg_data_filename(output_place_in + '_cADI_res_median.fits')
data = fits.getdata(dat, ext=0)

In [ ]:
Fig, ax = plt.subplots(1,1,figsize=(10,10))
plt.title(target, fontsize = '14')

im = ax.imshow(data[0], vmin = -1, vmax = 1, cmap = 'viridis')

plt.xlabel('Pixels', fontsize = '14')

plt.ylabel('Pixels', fontsize = '14')
plt.gca().invert_yaxis()

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = Fig.colorbar(im, ax=ax, label = 'Detector counts', cax=cax, cmap = 'viridis')
plt.show()

$\textbf{Finding positions by hand}$

In [ ]:
center = FitCenterModule(name_in = 'center', image_in_tag = 'cADI_res_median', fit_out_tag = 'PlanetCenter', 
                         guess = (, , 3, 3, , , 0))
#this module requires guesses, in order: x offset wrt centre, y offset wrt centre, Famplitude, position angle and amp offset. These can be 
#calculated by hand by taking the positions from the final reduced image opened in SAOImageDS9. This module fits a 2D Gaussian
#to the PSF of the point source and returns the best fit parameters in the Hdf5 database. These then need to be added by hand
#to the sepa, posa from calc_sep_pa3 by ways of appending.

In [ ]:
pipeline.add_module(center)
pipeline.run()